In [1]:
import pandas as pd
import numpy as np

In [2]:
def season_to_date(data):
    return np.int64(data[:2] + data[5:8])
    # eles fazem temporadas no formato 2000-01, no entanto só desejo saber um dos, como o outro padronizou como 
    # o ano final então manterei isso para a consistência dos dados

In [3]:
loc = "./Premier-League/"
EPL = pd.read_csv(loc + "EPLStandindsOriginal.csv")
EPLold = pd.read_csv(loc + "EPLStandindsOld.csv")

EPL = EPL[["Season", "Pos", "Team"]]
EPL["Season"] = EPL["Season"].apply(season_to_date)
EPL["Pos"] = EPL["Pos"]
print(EPL["Season"].unique())

In [4]:
df_result = pd.DataFrame()
df = EPL.copy()

df_result.index = df["Team"].unique()

# Agrupar os dados pelo time e criar as colunas para cada temporada com as respectivas posições
for season, group in df.groupby('Season'):
    df_result[season] = group.set_index('Team')['Pos']
    
# Reordenar as colunas em ordem crescente de temporada
print(df_result.shape)
df_result = df_result.reindex(sorted(df_result.columns), axis=1)
df_result = df_result.reset_index()

colunas_numericas = np.array(df_result.columns)[1:]

# df_result[colunas_numericas] = df_result[colunas_numericas].astype("int64")

colunas = df_result.columns[1:]
new_columns = {col: str(col) for col in colunas}
df_result = df_result.rename(columns=new_columns)

df_result = df_result.rename(columns={'index': 'Team2'})
df_result.columns

In [5]:
EPLold
# Preciso dar o Merge para pegar o ano de 1999-2000 que é utilizado
# Como os nomes podem ser diferentes e o nome do EPLold já tá padronizado com os nomes dos datasets
# Eu resolvi manter esses nomes e fazer o Merge checando as classificações
EPLold = EPLold.drop(EPLold.loc[EPLold["Team"] == "Wimbledon"].index)
EPLold = EPLold.drop(EPLold.loc[EPLold["Team"] == "Middlesboro"].index)

colunas = EPLold.columns[2:]
colunas = colunas.tolist()

EPLold.loc[EPLold["Team"] == "Fulham", "2007"] = 16.0
EPLold.loc[EPLold["Team"] == "Tottenham", "2004"] = 14.0
EPLold.loc[EPLold["Team"] == "Wigan", "2007"] = 17.0
EPLold.loc[EPLold["Team"] == "Reading", "2015"] = np.nan
EPLold.loc[EPLold["Team"] == "QPR", "2015"] = 20.0

# Aproveitei e corrigi alguns erros de posições que tinham no EPLold
# Confirmei esses erros com pesquisa então o EPLatual realmente estava correto

result = pd.merge(EPLold, df_result, on=colunas, how='outer')
# Aqueles times com a mesmas classificações de 2001 a 2016 se tornam um

result.loc[result["Team2"] == "Brighton & Hove Albion", "Team"] = "Brighton & Hove Albion"
result.loc[result["Team2"] == "Huddersfield Town", "Team"] = "Huddersfield Town"
result.loc[result["Team2"] == "Brentford", "Team"] = "Brentford"
# Padronizando alguns nomes que aparecerem apenas depois de 2016, ou seja não aparecem em EPLold

result  = result.drop("Team2", axis = 1)
result = result.sort_values(by="Team").reset_index(drop = True)

In [6]:
result.to_csv(loc + "EPLStandindsFinal.csv", index=False)